# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment

from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy, MedianStoppingPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice

import os
import shutil
import joblib

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'jobchangehyperml'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [3]:
cpu_cluster = "cpu-cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D12_V2',
                                                           max_nodes=5)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)

def clean_data(data):
    
    column_fillna = ['city', 'gender', 'relevent_experience',
       'enrolled_university', 'education_level', 'major_discipline',
       'experience', 'company_size', 'company_type', 'last_new_job']
    
    x_df = data.to_pandas_dataframe()
    x_df[column_fillna] = x_df[column_fillna].fillna('N_A')
    y_df = x_df.pop("target").astype('int')
    
    x_df = pd.get_dummies(x_df)

    return x_df, y_df

dataset = ws.datasets['traindata'] 

X, y = clean_data(dataset)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

model = LogisticRegression(max_iter=1000).fit(x_train, y_train)

accuracy = model.score(x_test, y_test)
run.log("accuracy", np.float(accuracy))
#accuracy

In [5]:
run.log("accuracy", np.float(accuracy), 'something')

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Columns: 194 entries, city_development_index to last_new_job_never
dtypes: float64(1), int64(1), uint8(192)
memory usage: 3.8 MB


## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [40]:
# Parameter sampler
ps = RandomParameterSampling({
        "--C": uniform(0.1,1.0),  #choice(1,2,3,4) uniform(0.1,1.0)
        "--max_iter": choice(1000, 1100, 1200, 1300)
    })

# Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval = 1)

#polcy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=0)

# SKLearn estimator for use with train.py
est = SKLearn(source_directory = '.', 
              compute_target = cpu_cluster, 
              entry_script = 'train.py')

# HyperDriveConfig using hyperparameter sampler, policy and estimator.
hyperdrive_config = HyperDriveConfig(estimator = est,
                                hyperparameter_sampling = ps,
                                policy = policy,
                                primary_metric_name = 'accuracy',
                                primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs = 10,
                                max_concurrent_runs = 4)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [41]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config, show_output=True) 

In [42]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_eeb2a541-01cf-4728-8ee3-306edb91164d
Web View: https://ml.azure.com/experiments/jobchangehyperml/runs/HD_eeb2a541-01cf-4728-8ee3-306edb91164d?wsid=/subscriptions/d4ad7261-832d-46b2-b093-22156001df5b/resourcegroups/aml-quickstarts-136651/workspaces/quick-starts-ws-136651

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-01-30T22:53:44.748045][API][INFO]Experiment created<END>\n""<START>[2021-01-30T22:53:45.309025][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n"<START>[2021-01-30T22:53:45.9199737Z][SCHEDULER][INFO]The execution environment is being prepared. Please be patient as it can take a few minutes.<END>"<START>[2021-01-30T22:53:45.746407][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"


ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [36]:
best_run = hyperdrive_run.get_best_run_by_primary_metric()

In [37]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
jobchangehyperml,HD_8a6d2cc4-19d3-46bd-a80b-1289a4fa541a_5,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [38]:
best_run.download_file('/outputs/model.joblib', 'hyperdrive_model.joblib')


In [39]:
#Register the best model
best_model_registered = best_run.register_model(model_name = 'best_hyperdrive_model', 
                                                model_path='outputs/model.joblib',
                                                description='best hyperdrive model')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.